In [1]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import json
import lz4.frame
import plot_world, plot_state
from Carbon_simulator.foundation import landmarks, resources
import seaborn as sns

num_steps = 500
n_agents = 5

In [2]:
logs = {}
cmaps = {}
for i in ["marl", "f_ge", "c_ge", "d_ge", "f_gml", "c_gml", "d_gml", "c", "d", "f"]:
    logs[i] = []
    cmaps[i] = []
    for j in range(5):
        filepath = "test/2/{}/dense_logs/logs_0{}/env-02.lz4".format(i, j)
        print("loading test/2/{}/dense_logs/logs_0{}/env-02.lz4".format(i, j))
        with lz4.frame.open(filepath, mode="rb", compression_level=16) as log_file:
            logs[i].append(json.loads(bytes.decode(log_file.readlines()[0])))
        states = logs[i][j]["states"][0]
        skills = {}
        for id in range(n_agents):
            skills[id] = {"Manufacture_volume": states[str(id)]['Manufacture_volume'],
                         "Research_ability": states[str(id)]['Research_ability']}
        skill1 = sorted(skills, key=lambda k: skills[k]['Manufacture_volume'])
        skill2 = sorted(skills, key=lambda k: skills[k]['Research_ability'])
        cmaps[i].append(sns.cubehelix_palette(rot=0.5, hue=2, reverse=True, as_cmap=False, n_colors=len(skill1) + 1)[1:])

loading test/2/marl/dense_logs/logs_00/env-02.lz4
loading test/2/marl/dense_logs/logs_01/env-02.lz4
loading test/2/marl/dense_logs/logs_02/env-02.lz4
loading test/2/marl/dense_logs/logs_03/env-02.lz4
loading test/2/marl/dense_logs/logs_04/env-02.lz4
loading test/2/f_ge/dense_logs/logs_00/env-02.lz4
loading test/2/f_ge/dense_logs/logs_01/env-02.lz4
loading test/2/f_ge/dense_logs/logs_02/env-02.lz4
loading test/2/f_ge/dense_logs/logs_03/env-02.lz4
loading test/2/f_ge/dense_logs/logs_04/env-02.lz4
loading test/2/c_ge/dense_logs/logs_00/env-02.lz4
loading test/2/c_ge/dense_logs/logs_01/env-02.lz4
loading test/2/c_ge/dense_logs/logs_02/env-02.lz4
loading test/2/c_ge/dense_logs/logs_03/env-02.lz4
loading test/2/c_ge/dense_logs/logs_04/env-02.lz4
loading test/2/d_ge/dense_logs/logs_00/env-02.lz4
loading test/2/d_ge/dense_logs/logs_01/env-02.lz4
loading test/2/d_ge/dense_logs/logs_02/env-02.lz4
loading test/2/d_ge/dense_logs/logs_03/env-02.lz4
loading test/2/d_ge/dense_logs/logs_04/env-02.lz4


In [3]:
logs.keys()

dict_keys(['marl', 'f_ge', 'c_ge', 'd_ge', 'f_gml', 'c_gml', 'd_gml', 'c', 'd', 'f'])

In [4]:
seed = 0
dict1 = {"Marl":"marl", 
         "Convex grandfathering":"c_ge", 
         "Decreasing grandfathering":"d_ge", 
         "Flat grandfathering":"f_ge",
         "Conex benchmarking":"c_gml",
         "Decreasing benchmarking":"d_gml",
         "Flat benchmarking":"f_gml",
         "Convex size indicator":"c",
         "Decreasing size indicator":"d",
         "Flat size indicator":"f"}
def plot(policy, step):
    log = logs[dict1[policy]][seed]
    cmap = cmaps[dict1[policy]][seed]
    world_map = plot_world.vis_world_frames(log, 1, step)
    skill_plot = plot_state.plot_skill(log, cmap)
    action_df = plot_state.extract_action(log, step)
    reward_plot = plot_state.plot_reward(log, step, cmap)
    qota_plot = plot_state.plot_quota(log, step, cmap)
    coin_plot = plot_state.plot_coins(log, step, cmap)
    level_plot = plot_state.plot_level(log, step, cmap)
    labor_plot = plot_state.plot_labor(log, step, cmap)
    trade_plot = plot_state.plot_trade(log, step, cmap)
    return world_map[0], skill_plot, action_df, reward_plot, qota_plot, coin_plot, level_plot, labor_plot, trade_plot

def change_seed(policy, step):
    global seed
    seed = (seed+1)%4
    log = logs[dict1[policy]][seed]
    cmap = cmaps[dict1[policy]][seed]
    world_map = plot_world.vis_world_frames(log, 1, step)
    skill_plot = plot_state.plot_skill(log, cmap)
    action_df = plot_state.extract_action(log, step)
    reward_plot = plot_state.plot_reward(log, step, cmap)
    qota_plot = plot_state.plot_quota(log, step, cmap)
    coin_plot = plot_state.plot_coins(log, step, cmap)
    level_plot = plot_state.plot_level(log, step, cmap)
    labor_plot = plot_state.plot_labor(log, step, cmap)
    trade_plot = plot_state.plot_trade(log, step, cmap)
    return world_map[0], skill_plot, action_df, reward_plot, qota_plot, coin_plot, level_plot, labor_plot, trade_plot

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("Carbon Market Simulator Dash Borad")
    with gr.Column():
        with gr.Row(equal_height=True):
            with gr.Column():
                radio = gr.Radio(["Marl", "Convex grandfathering", "Decreasing grandfathering", "Flat grandfathering", 
         "Conex benchmarking", "Decreasing benchmarking", "Flat benchmarking", 
                    "Convex size indicator", "Decreasing size indicator", "Flat size indicator"], 
                                label="Select allocation policy",
                                value="marl")
                name = gr.Slider(0, num_steps - 1, label="Select Step")
                with gr.Row():
                    play = gr.Button("Change seed")
            output3 = gr.Dataframe(label="Enterpises' actions")
        with gr.Row():
            output2 = gr.Plot(label="Skills", scale=1)
            output1 = gr.Plot(label="Grid world", scale=1)
            output9 = gr.Plot(label="Carbon price", scale=1)
            output8 = gr.Plot(label="Reward", scale=1)
        with gr.Row():
            output4 = gr.Plot(label="Carbon emission credits")
            output5 = gr.Plot(label="Coins")
            output6 = gr.Plot(label="Carbon emission level")
            output7 = gr.Plot(label="Labor")
                
    name.change(fn=plot, inputs=[radio, name], outputs=[output1, output2, output3, output8, output4, output5, output6, output7, output9])
    radio.change(fn=plot, inputs=[radio, name], outputs=[output1, output2, output3, output8, output4, output5, output6, output7, output9])
    play.click(fn=change_seed, inputs=[radio, name], outputs=[output1, output2, output3, output8, output4, output5, output6, output7, output9])      
 
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
